In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pathlib import Path

In [2]:
# Diretórios com outputs
output_dir = Path('../SCFs_rnd.out/')

In [3]:
with open(file=output_dir / 'energias.dat', mode='r') as f:
    linhas = f.readlines()

In [ ]:
arquivos = sorted(output_dir.glob('scf-*.out')) # : list[PosixPath()]

if len(arquivos) != len(linhas):
    raise ValueError("Nº linhas em energias.dat diferente do nº de inputs")

In [5]:
dados = [] # list[Dict] para DataFrame

for linha, arquivo in zip(linhas, arquivos):
    match_energia = re.search(r'=\s+(-?\d+\.\d+)', linha)
    
    # Energia dos arquivos.
    energia = float(match_energia.group(1)) if match_energia else None
    
    # Parâmetros estruturais via nome do arquivo
    match_nome = re.match(r'scf-([0-9.]+)-([0-9.]+)-(\d+)\.out', arquivo.name)
    if match_nome:
        a = float(match_nome.group(1))
        ratio_ca = float(match_nome.group(2))
        num_estrutura = int(match_nome.group(3))
    else:
        raise ValueError(f'Nome de arquivo não reconhecido: {arquivo.name}')
    
    dados.append(
        {
        'Energia Total (Ry)': energia,
        'a': a,
        'ratio c/a': ratio_ca,
        'num_estrutura': num_estrutura,
        'nome_arquivo': arquivo.name
        }
        )


In [6]:
df = pd.DataFrame(dados)
df

In [7]:
# Agrupando por estrutura
df1 = df[df['num_estrutura'] == 1]
df2 = df[df['num_estrutura'] == 2]
df3 = df[df['num_estrutura'] == 3]
df4 = df[df['num_estrutura'] == 4]
df5 = df[df['num_estrutura'] == 5]

lista_dfs = [df1, df2, df3,df4,df5]

In [8]:
estruturas = df['num_estrutura'].unique()
estruturas

In [10]:
fig, axes = plt.subplots(
    nrows=5, 
    ncols=1,
    figsize=(10,15),
    sharey=True
)
 # [1,2,3,4,5]

for df, estrutura in zip(lista_dfs, estruturas):
    
    idx_axes_estrutura = estrutura-1 
    axes[idx_axes_estrutura].scatter(df.index, df['Energia Total (Ry)'])
    axes[idx_axes_estrutura].set_title(f'Estrutura {idx_axes_estrutura+1}')


fig.suptitle('Energia em função de (a, c/a) para cada estrutura', fontsize=18)
plt.tight_layout()
plt.show()